In [1]:
# Standard imported packages
import numpy as np  
import symengine as sp
from matplotlib import pyplot as plt

# Tools from Bocop's Python API
from BocopUtils import readSolFile

# Tools from our
from symengine_function import gradient, lie_bracket, xp_bracket
from shooting import shooting
#from sympy.parsing.sympy_parser import parse_expr